In [1]:

import pathlib 
import optuna
import pandas as pd
import joblib
import torch
import sys

from sklearn.decomposition import PCA, FastICA, NMF
from torch.utils.data import DataLoader, TensorDataset

script_directory = pathlib.Path("../2.train-VAE/utils/").resolve()
sys.path.insert(0, str(script_directory))
from betavae import BetaVAE, train_vae
from betatcvae import BetaTCVAE, train_tc_vae
from vanillavae import VanillaVAE, train_vvae
from optimize_utils import get_optimize_args, objective, get_optimizer
from optimize_utils_tcvae import get_optimize_args_tc, objective_tc, get_optimizer_tc
from optimize_utils_vvae import get_optimize_args_vvae, objective_vvae, get_optimizer_vvae

script_directory = pathlib.Path("../utils/").resolve()
sys.path.insert(0, str(script_directory))
from data_loader import load_train_test_data, load_model_data

In [2]:
# Load command line arguments
args = get_optimize_args()
tc_args = get_optimize_args_tc()
vvae_args = get_optimize_args_vvae()

# Load data
data_directory = pathlib.Path("../0.data-download/data").resolve()

train_df, test_df, val_df, load_gene_stats = load_train_test_data(
    data_directory, train_or_test="all", load_gene_stats=True, zero_one_normalize=True
)
train_data = pd.DataFrame(train_df)

dependency_file = pathlib.Path(f"{data_directory}/CRISPRGeneEffect.parquet").resolve()
gene_dict_file = pathlib.Path(f"{data_directory}/CRISPR_gene_dictionary.parquet").resolve()
dependency_df, gene_dict_df= load_model_data(dependency_file, gene_dict_file)
gene_dict_df = pd.DataFrame(gene_dict_df)
train_data.head()

(1150, 18444)


,0,1,2,3,4,5,6,7,8,9,...,2708,2709,2710,2711,2712,2713,2714,2715,2716,2717
0,0.594385,0.519079,0.427987,0.688098,0.209469,0.325300,0.503507,0.376132,0.638695,0.786441,...,0.000000,0.247000,0.396753,0.254780,0.505385,0.324102,0.402266,0.532244,0.733186,0.376822
1,0.717219,0.501631,0.298584,0.795552,0.780744,0.191839,0.578993,0.497818,0.509780,0.814088,...,0.181179,0.390783,0.692250,0.571378,0.697221,0.346431,0.379257,0.453511,0.099317,0.520031
2,0.582346,0.553260,0.622962,0.707061,0.778673,0.427394,0.592183,0.544953,0.576460,0.800464,...,0.459704,0.291439,0.561333,0.441115,0.718143,0.431680,0.326140,0.276138,0.485595,0.647104
3,0.477536,0.758997,0.559317,0.479743,0.886465,0.331715,0.589289,0.678770,0.610409,0.732688,...,0.233434,0.534838,0.729337,0.666914,0.925434,0.599512,0.719212,0.452738,0.507295,0.434686
4,0.716245,0.591769,0.480939,0.311575,0.382095,0.611383,0.588281,0.568117,0.363668,0.714433,...,0.480942,0.448338,0.412759,0.557914,0.519348,0.326683,0.689145,0.375009,0.313097,0.615826


In [3]:
# Convert dataframes to tensors
train_tensor = torch.tensor(train_df, dtype=torch.float32)
test_tensor = torch.tensor(test_df, dtype=torch.float32)
val_tensor = torch.tensor(val_df, dtype=torch.float32)

In [4]:
# Directory where models will be saved
model_save_dir = pathlib.Path("saved_models")
model_save_dir.mkdir(parents=True, exist_ok=True)

# Define the optimization process for the models
latent_dims = [2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 16, 18, 20, 25, 30, 35, 40, 45, 50, 60, 70, 80, 90, 100, 150, 200]
model_names = ["pca", "ica", "nmf", "vanillavae", "betavae", "betatcvae"]

for num_components in latent_dims:
    for model_name in model_names:
        model = None  # Initialize model as None for each iteration
        
        if model_name in ["pca", "ica", "nmf"]:
            if model_name == "pca":
                model = PCA(n_components=num_components)
            elif model_name == "ica":
                model = FastICA(n_components=num_components)
            elif model_name == "nmf":
                model = NMF(n_components=num_components, init='nndsvd', max_iter=2000, random_state=0)
            
            # Fit model to data
            model.fit(train_data)
        
        elif model_name == "betavae":
            study = optuna.create_study(direction="minimize")
            study.optimize(lambda trial: objective(trial, train_tensor, train_tensor, train_data, latent_dim=num_components), n_trials=50)
            best_trial = study.best_trial
            model = BetaVAE(input_dim=train_data.shape[1], latent_dim=num_components, beta=best_trial.params['beta'])
            train_loader = DataLoader(TensorDataset(train_tensor), batch_size=best_trial.params['batch_size'], shuffle=True)
            optimizer = get_optimizer(best_trial.params['optimizer_type'], model.parameters(), best_trial.params['learning_rate'])
            train_vae(model, train_loader, optimizer, best_trial.params['epochs'])
        
        elif model_name == "betatcvae":
            study = optuna.create_study(direction="minimize")
            study.optimize(lambda trial: objective_tc(trial, train_tensor, train_tensor, train_data, latent_dim=num_components), n_trials=50)
            best_trial = study.best_trial
            model = BetaTCVAE(input_dim=train_data.shape[1], latent_dim=num_components, beta=best_trial.params['beta'])
            train_loader = DataLoader(TensorDataset(train_tensor), batch_size=best_trial.params['batch_size'], shuffle=True)
            optimizer = get_optimizer_tc(best_trial.params['optimizer_type'], model.parameters(), best_trial.params['learning_rate'])
            train_tc_vae(model, train_loader, optimizer, best_trial.params['epochs'])

        elif model_name == "vanillavae":
            study = optuna.create_study(direction="minimize")
            study.optimize(lambda trial: objective_vvae(trial, train_tensor, train_tensor, train_data, latent_dim=num_components), n_trials=50)
            
            best_trial = study.best_trial
            model = VanillaVAE(input_dim=train_data.shape[1], latent_dim=num_components)
            train_loader = DataLoader(TensorDataset(train_tensor), batch_size=best_trial.params['batch_size'], shuffle=True)
            optimizer = get_optimizer_vvae(best_trial.params['optimizer_type'], model.parameters(), best_trial.params['learning_rate'])
            train_vvae(model, train_loader, optimizer, best_trial.params['epochs'])

        # Save the trained model with joblib
        if model:
            model_filename = model_save_dir / f"{model_name}_{num_components}_components_model.joblib"
            joblib.dump(model, model_filename)
            print(f"Saved {model_name} with {num_components} components to {model_filename}")

Saved pca with 2 components to saved_models/pca_2_components_model.joblib
Saved ica with 2 components to saved_models/ica_2_components_model.joblib


/home/juliacurd/anaconda3/envs/gene_dependency_representations/lib/python3.12/site-packages/sklearn/decomposition/_nmf.py:1759: ConvergenceWarning: Maximum number of iterations 2000 reached. Increase it to improve convergence.
  warnings.warn(
[I 2024-09-23 15:30:25,352] A new study created in memory with name: no-name-c9f5082d-af56-464b-a632-0d7ef96c9747


Saved nmf with 2 components to saved_models/nmf_2_components_model.joblib


[I 2024-09-23 15:30:27,781] Trial 0 finished with value: 47.06141193446828 and parameters: {'learning_rate': 0.0033267768007463317, 'batch_size': 96, 'epochs': 142, 'optimizer_type': 'rmsprop'}. Best is trial 0 with value: 47.06141193446828.
[I 2024-09-23 15:30:54,143] Trial 1 finished with value: 46.93966938986707 and parameters: {'learning_rate': 0.0009812305032814337, 'batch_size': 36, 'epochs': 822, 'optimizer_type': 'adam'}. Best is trial 1 with value: 46.93966938986707.
[I 2024-09-23 15:31:19,810] Trial 2 finished with value: 47.001069481693094 and parameters: {'learning_rate': 0.0017504425150852764, 'batch_size': 87, 'epochs': 650, 'optimizer_type': 'rmsprop'}. Best is trial 1 with value: 46.93966938986707.
[I 2024-09-23 15:31:33,834] Trial 3 finished with value: 46.99592649545242 and parameters: {'learning_rate': 0.003442857815154822, 'batch_size': 107, 'epochs': 331, 'optimizer_type': 'adam'}. Best is trial 1 with value: 46.93966938986707.
[I 2024-09-23 15:32:12,339] Trial 4 f

Saved vanillavae with 2 components to saved_models/vanillavae_2_components_model.joblib


...

[I 2024-09-25 13:27:17,489] Trial 0 finished with value: 47.23146025244869 and parameters: {'learning_rate': 0.0036137542958266542, 'batch_size': 69, 'epochs': 121, 'optimizer_type': 'rmsprop'}. Best is trial 0 with value: 47.23146025244869.
[I 2024-09-25 13:27:37,937] Trial 1 finished with value: 47.00798594916045 and parameters: {'learning_rate': 0.0006554880397234315, 'batch_size': 42, 'epochs': 436, 'optimizer_type': 'rmsprop'}. Best is trial 1 with value: 47.00798594916045.
[I 2024-09-25 13:28:10,772] Trial 2 finished with value: 46.765473414179105 and parameters: {'learning_rate': 0.001659456894121643, 'batch_size': 51, 'epochs': 703, 'optimizer_type': 'adam'}. Best is trial 2 with value: 46.765473414179105.
[I 2024-09-25 13:28:25,827] Trial 3 finished with value: 46.79716177413713 and parameters: {'learning_rate': 0.0014049619635988736, 'batch_size': 88, 'epochs': 430, 'optimizer_type': 'adam'}. Best is trial 2 with value: 46.765473414179105.
[I 2024-09-25 13:28:49,238] Trial 4 

Saved vanillavae with 150 components to saved_models/vanillavae_150_components_model.joblib


[I 2024-09-25 13:49:35,338] Trial 0 finished with value: 47.634466097248136 and parameters: {'beta': 7.471198571043939, 'learning_rate': 0.0027534794303842682, 'batch_size': 67, 'epochs': 668, 'optimizer_type': 'adam'}. Best is trial 0 with value: 47.634466097248136.
[I 2024-09-25 13:49:36,600] Trial 1 finished with value: 149.22559450064134 and parameters: {'beta': 1.8089974894821625, 'learning_rate': 0.0018777980348671142, 'batch_size': 55, 'epochs': 21, 'optimizer_type': 'adam'}. Best is trial 0 with value: 47.634466097248136.
[I 2024-09-25 13:49:41,356] Trial 2 finished with value: 110.65882039995336 and parameters: {'beta': 2.569237632714896, 'learning_rate': 0.00023150746646367147, 'batch_size': 62, 'epochs': 112, 'optimizer_type': 'rmsprop'}. Best is trial 0 with value: 47.634466097248136.
[I 2024-09-25 13:49:43,628] Trial 3 finished with value: 132.99779335135844 and parameters: {'beta': 9.831596027382538, 'learning_rate': 0.0008972994900980621, 'batch_size': 95, 'epochs': 59, 

Saved betavae with 150 components to saved_models/betavae_150_components_model.joblib


[I 2024-09-25 14:09:48,160] Trial 0 finished with value: 44.046698316887245 and parameters: {'beta': 1.2258296999179377, 'learning_rate': 7.810556817110252e-05, 'batch_size': 73, 'epochs': 907, 'optimizer_type': 'rmsprop'}. Best is trial 0 with value: 44.046698316887245.
[I 2024-09-25 14:09:54,326] Trial 1 finished with value: 49.95222505028568 and parameters: {'beta': 1.1483377805578958, 'learning_rate': 3.927177463417574e-05, 'batch_size': 66, 'epochs': 73, 'optimizer_type': 'adam'}. Best is trial 0 with value: 44.046698316887245.
[I 2024-09-25 14:10:23,554] Trial 2 finished with value: 42.91328876552297 and parameters: {'beta': 1.7215565102732624, 'learning_rate': 8.237725023395433e-05, 'batch_size': 83, 'epochs': 315, 'optimizer_type': 'adam'}. Best is trial 2 with value: 42.91328876552297.
[I 2024-09-25 14:11:51,355] Trial 3 finished with value: 38.34333641849347 and parameters: {'beta': 1.80509523585338, 'learning_rate': 9.94173252577983e-05, 'batch_size': 90, 'epochs': 946, 'opt